# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [26]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [32]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Sit', 'Crouch', 'Belly', 'StandInit', 'Back', 'Right', 'Left', 'Knee', 'HeadBack', 'Frog', 'Stand']


In [54]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    # TODO: what is left to do here???
        
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    print(filename)
    data = pickle.load(open(filename, "rb"))
    target = [i] * len(data)
    return data, target

In [183]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i,_ in enumerate(classes):
    d, t = load_pose_data(i)
    all_data.extend(d)
    all_target.extend(t)
    
# all_data = np.array(all_data)
# all_target = np.array(all_target)
print('total number of data', len(all_data))
# print(all_data.shape)

total number of data 222


In [184]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [185]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [186]:
# YOUR CODE HERE
clf.fit(np.asarray(all_data)[training_data], 
        np.asarray(all_target)[training_data])


SVC(C=100.0, gamma=0.001)

### predicting

In [198]:
# original is not working:
# clf.predict(all_data[-1]), all_target[-1]

# this is
clf.predict(np.asarray(all_data)[training_data])


array([ 1,  0,  0,  3,  5,  1,  1,  3,  9,  1,  6,  3, 10,  3,  3,  4,  7,
        1,  4,  3,  7,  0,  2,  3,  7,  4,  6,  0,  4,  0,  6,  2,  3,  3,
        0, 10,  2,  7,  3,  1,  1,  6,  6,  2,  3,  6,  1,  0,  1,  8,  9,
        5,  3,  3,  2,  0,  4,  3,  6,  3,  3,  3,  0,  5,  0,  0,  3,  1,
        4,  8,  2,  6,  3,  6, 10,  4,  0,  2,  7,  8, 10,  0, 10,  0,  4,
        4,  3,  2,  2,  7,  3,  0,  3,  7,  9, 10,  6, 10,  2,  9,  0,  5,
        2,  3,  0,  6,  2,  5,  4,  7,  3,  6,  3,  1,  6,  3,  4,  1,  1,
        0,  6,  8,  7,  0,  3,  3,  5,  9,  1,  7,  2,  1,  1,  1,  3,  3,
        1,  6,  4,  0,  3,  3,  1,  9,  0,  6,  5,  4,  3,  3,  1,  4,  4,
        3,  3])

In [199]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [200]:
expected = []
predicted = []
# YOUR CODE HERE
expected = np.asarray(all_target)[training_data]
predicted = clf.predict(np.asarray(all_data)[training_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       0.95      1.00      0.97        19
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        36
           4       1.00      1.00      1.00        15
           5       1.00      0.88      0.93         8
           6       1.00      1.00      1.00        16
           7       1.00      1.00      1.00        10
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         7

    accuracy                           0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[21  0  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0 13  0  0  0  0  0  0  0  0]
 [ 0  0  0 36  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [201]:
expected = []
predicted = []
# YOUR CODE HERE
testing_data = permutation[n_training_data:]
expected = np.asarray(all_target)[testing_data]
predicted = clf.predict(np.asarray(all_data)[testing_data])

# evaluate(expected, predicted)
len(testing_data) + len(training_data)

222

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [202]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

TypeError: write() argument must be str, not bytes

Then, in the application we can load the trained classifier again.

In [203]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1]), all_target[-1]

TypeError: a bytes-like object is required, not 'str'